In [1]:
import feedparser
import pandas as pd
import gensim
from gensim.models import Word2Vec
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
stop_words = stopwords.words('english')
from pyemd import emd
from scipy.cluster.hierarchy import fclusterdata
from sklearn.cluster import DBSCAN, AffinityPropagation, AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier, kneighbors_graph
import pickle
import time
import hashlib
from sklearn.exceptions import NotFittedError

#from itertools import combinations
#from tqdm import tqdm_notebook
#from scipy.stats import skew, kurtosis
#from sklearn.preprocessing import StandardScaler
#from sklearn.decomposition import PCA, TruncatedSVD, LatentDirichletAllocation

bbc_rss = ['http://feeds.bbci.co.uk/news/rss.xml', 
           'http://feeds.bbci.co.uk/news/world/rss.xml', 
           'http://feeds.bbci.co.uk/news/uk/rss.xml', 
           'http://feeds.bbci.co.uk/news/business/rss.xml', 
           'http://feeds.bbci.co.uk/news/politics/rss.xml', 
           'http://feeds.bbci.co.uk/news/health/rss.xml', 
           'http://feeds.bbci.co.uk/news/education/rss.xml', 
           'http://feeds.bbci.co.uk/news/science_and_environment/rss.xml', 
           'http://feeds.bbci.co.uk/news/technology/rss.xml', 
           'http://feeds.bbci.co.uk/news/entertainment_and_arts/rss.xml']

In [2]:
class Embeddings:
    
    def __init__(self, create=False, ser_model_path='W2VModel',
                 embeddings='GoogleNews-vectors-negative300.bin.gz',
                 model_fun=gensim.models.KeyedVectors.load_word2vec_format, binary=True, norm=True):
        self.ser_model = ser_model_path
        self.embeddings = embeddings
        self.model_fun = model_fun
        self.binary = binary
        self.norm = norm
        
        if create == False:
            self.model = self.load_model()
        else:
            self.model = self.create_model()
        
            
    def create_model(self):
        model = self.model_fun(self.embeddings, binary=self.binary)
        if self.norm:
            model.init_sims(replace=True)
        return model
            
    def load_model(self):
        with open(self.ser_model, 'rb') as file:
            model = pickle.load(file)
        return model

In [3]:
# ? TODO: dist_matrix(x,y) from two news vectors (if we want some classification based only on distances, + use wmd!)
class News_Vectorizer:
    
    def __init__(self, model, news=None):
        self.news = news #array of strings
        self.model = model #Word2Vec model
        if self.news is not None:
            self.news_vectors = self.news2vec(self.news) #vector representations
        else:
            self.news_vectors = None
        self.cos_dist = None #cosine distance matrix
        self.wm_dist = None #wmd-matrix
    
    def wmd(self, q1, q2):
        q1 = str(q1).lower().split()
        q2 = str(q2).lower().split()
        q1 = [w for w in q1 if w not in stop_words]
        q2 = [w for w in q2 if w not in stop_words]
        return self.model.wmdistance(q1, q2)
    
    def sent2vec(self, s):
        words = str(s).lower()
        words = word_tokenize(words)
        words = [w for w in words if not w in stop_words]
        words = [w for w in words if w.isalpha()]
        M = []
        for w in words:
            try:
                M.append(self.model[w])
            except:
                continue
        M = np.array(M)
        v = M.sum(axis=0)
        return v / np.sqrt((v ** 2).sum())
    
    def news2vec(self, news):
        # update self.news, self.news_vectors
        news_vectors = np.array([self.sent2vec(text) for text in news])
        self.news = news
        self.news_vectors = news_vectors
        return news_vectors
    
    def dist_vec(self, news_item, news=None, metric='cosine'):
        #computes distances between given item and news (or self.news)
        news_item = self.sent2vec(news_item)
        if news is not None:
            news = self.news2vec(news)
        else:
            news = self.news_vectors
        if news is None:
            return 'no news to compute distances'
        if metric == 'cosine':
            dist_vec = np.array([cosine(news_item, i) for i in news])
        elif metric == 'wmd':
            dist_vec = np.array([self.wmd(news_item, i) for i in news])
        return dist_vec
    
    def cosine_matrix(self): 
        cdist = np.zeros((len(self.news_vectors), len(self.news_vectors)))
        for n, i in enumerate(self.news_vectors):
            for m, j in enumerate(self.news_vectors):
                cdist[n, m] = cosine(i, j)
        self.cos_dist = cdist
        return cdist
    
    def wmd_matrix(self): #list (news)
        wmdist = np.zeros((len(self.news), len(self.news)))
        for n, i in enumerate(self.news):
            for m, j in enumerate(self.news):
                wmdist[n, m] = self.wmd(i, j)
        self.wm_dist = wmdist
        return wmdist

In [4]:
class RSS_Feeds:
    
    def __init__(self, urls):
        self.urls = urls
        self.feeds = self.get_feeds()
        self.df_news = self.create_df()
        self.df_unique_news = self.create_unique()
        
    def get_feeds(self):
        return [feedparser.parse(feed) for feed in self.urls]
    
    def get_category(self, feed):
        # sources may have different category names - agg categories?
        return feed.feed.get('title', '')

    def get_title_summary(self, feed, sep='. '): #get and join title and summary for each entry in feed
        titles = [entry['title'] for entry in feed['entries']]
        summaries = [entry['summary'] for entry in feed['entries']]
        title_summary = [entry['title'] + sep + entry['summary'] for entry in feed['entries']]
        return titles, summaries, title_summary
    
    def get_date(self, feed): #(year, month, day) for each entry in feed
        return([entry['published_parsed'][:3] for entry in feed['entries']])
    
    def get_time(self, feed): #(hour, min, sec) for each entry in feed
        return([entry['published_parsed'][3:6] for entry in feed['entries']])
    
    def get_datetime_nparsed(self, feed): #not parsed date and time for each entry in feed
        return([entry['published'] for entry in feed['entries']])
    
    def get_link(self, feed): # link for each entry in feed
        return([entry['link'] for entry in feed['entries']])
    
    def str2hash(self, s):
        return hashlib.md5(s.encode()).hexdigest()
    
    def create_df(self): 
        news, title, summary, category, pdate, ptime, fdatetime, links  = [], [], [], [], [], [], [], []
        for feed in self.feeds:
            cat = self.get_category(feed)
            titles, summaries, texts = self.get_title_summary(feed)
            d_ymd, t_hms = self.get_date(feed), self.get_time(feed)
            fdt = self.get_datetime_nparsed(feed)
            news_links = self.get_link(feed)
            
            cat = np.resize([cat], len(texts))
            news.extend(texts)
            title.extend(titles)
            summary.extend(summaries)
            pdate.extend(d_ymd)
            ptime.extend(t_hms)
            fdatetime.extend(fdt)
            links.extend(news_links)
            category.extend(cat)
        df_news = pd.DataFrame({'news':news, 
                                'category':category,
                                'title':title, 
                                'summary':summary,
                                'link':links,
                                'date':pdate, 
                                'time':ptime, 
                                'datetime':fdatetime})
        df_news['ID'] = df_news.news.apply(self.str2hash)
        self.df_news = df_news
        return df_news
    
    def create_unique(self):
        df_unique_news = self.df_news.groupby('news').agg({'category':list, 
                                                           'title': np.unique, 
                                                           'summary': np.unique, 
                                                           'link': np.unique, 
                                                           'date': np.unique, 
                                                           'time': np.unique, 
                                                           'datetime': np.unique, 
                                                           'ID': np.unique})
        df_unique_news.reset_index(inplace=True)
        self.df_unique_news = df_unique_news
        return df_unique_news
    
    def get_unique_news(self):
        return self.df_unique_news.news.values

In [5]:
class Aggregator:
    
    def __init__(self, clusterizer, classifier, labeled_data=None, labels=None, clust_weights=None):
        self.clusterizer = clusterizer
        self.classifier = classifier
        self.labeled_data = labeled_data #already clustered viewed, vector representations as ndarray
        self.labels = labels #clust nums of labeled_data, ndarray
        self.clust_weights = clust_weights # DataFrame, colnames=['clust', 'weight']
        
    def clusterize(self, data):
        labels = self.clusterizer.fit_predict(data)
        return data, labels
    
    def classify(self, new_data): #if one sample: reshape sent2vec output to (1, 300)
        try:
            predicted = self.classifier.predict(new_data)
        except NotFittedError as e:
            return(repr(e))
        return predicted
    
    def fit_classifier(self):
        X, y = self.labeled_data, self.labels
        self.classifier.fit(X, y)
        return self.classifier
    
    def prep_data(self, new_data=None):
        if self.labeled_data is None and new_data is None:
            return None
        else:
            try:
                ldata = pd.DataFrame(self.labeled_data)
            except:
                ldata = None
            try:
                ndata = pd.DataFrame(new_data)
            except:
                ndata=None
            try:
                data = pd.concat([ldata, ndata]).values
                return data
            except:
                return None
    
    def update_weights(self): #sum weights = 1 required in News_Finder
        unique, counts = np.unique(self.labels[self.labels != -1], return_counts=True)
        weights = counts/counts.sum() #smth like this
        weights = np.asarray((unique, weights)).T # [label, weight]
        self.clust_weights = pd.DataFrame({'clust': weights[:,0].astype(int), 'weight': weights[:,1]})
        return self.clust_weights
    
    def update_aggregator(self, new_data):
        data = self.prep_data(new_data=new_data)
        if data is None:
            return 'no data'
        else:
            self.labeled_data, self.labels = self.clusterize(data)    
            self.fit_classifier()
            self.update_weights()
            return 'updated'
        

In [6]:
class News_Finder():
    
    def __init__(self, df_news, news_vectorizer): #df_news: DF with non-viewed news items; News_Vectorizer instance
        self.df_news = df_news
        self.df_unique_news = self.create_unique()
        self.news_vectorizer = news_vectorizer
        
    def update_news(self, df_news):
        self.df_news = df_news
        self.df_unique_news = self.create_unique()
        return 'updated'
    
    def create_unique(self):
        df_unique_news = self.df_news.groupby('ID').agg({'news': np.unique, 
                                                         'category':list, 
                                                         'title': np.unique, 
                                                         'summary': np.unique, 
                                                         'link': np.unique, 
                                                         'date': np.unique, 
                                                         'time': np.unique, 
                                                         'datetime': np.unique})
        df_unique_news.reset_index(inplace=True)
        return df_unique_news
    
    def get_from_categories(self, n=5):
        # returns n top news from each category
        return self.df_news.groupby('category').head(n) 
    
    def get_similar(self, news_item_ID, metric='cosine', n=5):
        # returns the n most similar news to news_item
        all_news = self.df_unique_news.query('ID != @news_item_ID').copy()
        news_item = self.df_unique_news.query('ID == @news_item_ID').copy()
        dist_vec = self.news_vectorizer.dist_vec(news_item.news, all_news.news.values, metric=metric)
        all_news['dist'] = dist_vec
        return all_news.nsmallest(n, 'dist')
    
    def get_interesting(self, aggregator, n=20): #fitted Aggregator instance for classification & weights
        # TODO: return n news
        # TODO: if there are not enough news in clusters (for weights), return another news?
        all_news = self.df_unique_news.copy()
        news_vec = self.news_vectorizer.news2vec(all_news.news.values)
        #print(news_vec.shape)
        weights = aggregator.clust_weights
        labels = aggregator.classify(news_vec)
        all_news['label'] = labels
        all_news = all_news.query('label != -1')
        # TODO: filter 'outlier' cluster
        # ?TODO: distances to choose the most relevant items in cluster 
        n_from_cluster = np.ceil((aggregator.clust_weights.weight*n)).astype(int)
        dflist = []
        for cluster, n in zip(weights.clust, n_from_cluster):
            dflist.append(all_news.query('label == @cluster').head(n))
        interesting = pd.concat(dflist)
        return interesting

In [7]:
class Data_Manager:
    
    def __init__(self, path_dict=None): #path_dict {'csv':{obj:path}, 'serialized':{obj:path}}
        self.path_dict = path_dict
        if self.path_dict is not None:
            self.data_dict = self.load_data()
        else:
            self.data_dict = {}
        
    def load_data(self):
        data_dict = {}
        try:
            for obj_name, path in self.path_dict['csv'].items():
                data_dict[obj_name] = pd.read_csv(path, index_col=0)
        except:
            print('something is not ok with "csv" key or it does not exist')
        try:
            for obj_name, path in self.path_dict['serialized'].items():
                with open(path, 'rb') as file:
                    data_dict[obj_name] = pickle.load(file)
        except:
            print('something is not ok with "serialized" key or it does not exist')
        return data_dict
    
    def delete_old(self, obj_name, n_recent=100):
        #del old data, except n_recent
        data = self.get_data_item(obj_name)
        if type(data) == pd.core.frame.DataFrame and data.shape[0] > n_recent:
            data = data.tail(n_recent)
            self.update_data_item(obj_name, data, concat=False)
            return('old entries removed')
        return('not enough entries to delete or is not DF')
    
    #def prep_data(self):
    #    #maybe some data manipulations
    #    pass
    
    def get_data_item(self, obj_name):
        return self.data_dict.get(obj_name, 'Does not exist')
    
    def update_data_item(self, obj_name, new_data, concat=False): #concat [True, False] - if concat data
        if concat == False:
            self.data_dict[obj_name] = new_data
            return('upd: set data_dict[obj] = new_data')
        elif concat == True:
            if obj_name in self.data_dict.keys():
                data = self.get_data_item(obj_name)
                if type(data) == pd.core.frame.DataFrame:
                    try:
                        data = pd.concat([data, new_data], sort=False)
                        self.data_dict[obj_name] = data
                        return 'updated'
                    except:
                        return 'could not update'
                elif type(data) == np.ndarray:
                    try:
                        data = np.vstack([data, new_data])
                        self.data_dict[obj_name] = data
                        return 'updated'
                    except:
                        return 'could not update'
                else:
                    self.data_dict[obj_name] = new_data
                    return 'upd: obj = new_data (not an array or DF)'
            else:
                self.data_dict[obj_name] = new_data
                return 'upd: obj = new_data (obj did not exist yet)'
    
    def save_model(self, data_items='all'): #data_items: 'all' or list of keys for data_dict
        if data_items == 'all':
            data_items = self.data_dict.keys()
        for obj_name in data_items:
            data = self.data_dict[obj_name]
            if type(data) == pd.core.frame.DataFrame:
                data.to_csv(obj_name + '.csv')
            elif type(data) == np.ndarray:
                # are there any ndarrays?..
                # TODO: write csv...
                pass
            else:
                with open(obj_name, 'wb') as file:
                    pickle.dump(data, file)
        return 'saved'

### Schnittstelle-Klasse zu GUI

(volle Funktionalität, erfordert alle oben definierten Klassen)

In [8]:
class Communicator:
    '''Application start -> initialize Communicator instance and call `start()`
    to start an existing system or create a new system and get DataFrames, e.g.:
    
    `CI = Communicator()
    news_in_categories, interesting_news = CI.start()
    display(news_in_categories)`
    
    Then use `handle_input()` to process user input and get system output, e.g.:
    
    `news_in_categories, interesting_news = CI.handle_input(u_input='upd')` '''   
    
    def __init__(self, feeds='default', model='default', path_dict='default', data_items_names='default', 
                 n_from_cats=10, n_interesting=20, n_similar=5, THRESHOLD=20):
        
        if feeds == 'default':
            self.feeds = ['http://feeds.bbci.co.uk/news/rss.xml', 
                          'http://feeds.bbci.co.uk/news/world/rss.xml', 
                          'http://feeds.bbci.co.uk/news/uk/rss.xml', 
                          'http://feeds.bbci.co.uk/news/business/rss.xml', 
                          'http://feeds.bbci.co.uk/news/politics/rss.xml', 
                          'http://feeds.bbci.co.uk/news/health/rss.xml', 
                          'http://feeds.bbci.co.uk/news/education/rss.xml', 
                          'http://feeds.bbci.co.uk/news/science_and_environment/rss.xml', 
                          'http://feeds.bbci.co.uk/news/technology/rss.xml', 
                          'http://feeds.bbci.co.uk/news/entertainment_and_arts/rss.xml']
        else:
            self.feeds = feeds
            
        if model == 'default':
            self.model = Embeddings().model
        else:
            self.model = model
        
        if path_dict == 'default':
            self.path_dict = {'csv':{'df_viewed':'df_viewed.csv', 
                                     'df_labeled':'df_labeled.csv', 
                                     'clust_weights':'clust_weights.csv'}, 
                              'serialized':{'classifier':'classifier', 
                                            'clusterizer':'clusterizer'}}
        else:
            self.path_dict = path_dict
            
        if data_items_names == 'default':
            self.data_items_names = ['df_viewed', 'df_labeled', 'clust_weights', 'classifier', 'clusterizer']
        else:
            self.data_items_names = data_items_names
        
        self.STATE = None
        self.n_from_cats = n_from_cats
        self.n_interesting = n_interesting
        self.n_similar = n_similar
        self.THRESHOLD = THRESHOLD
        
        self.DM = None
        self.NFind = None
        self.AGG = None
        
        self.RSS = RSS_Feeds(self.feeds)
        self.NVec = News_Vectorizer(model=self.model)
        
        self.init_classification_model = RadiusNeighborsClassifier(radius=0.49, weights='distance', 
                                                                   metric='cosine', outlier_label='-1')
        self.init_clustering_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', 
                                                             linkage='complete', distance_threshold=0.65)

        
        
    
    def create_new_system(self):
        # create a new system without any already existing data
        self.DM = Data_Manager()
        self.AGG = Aggregator(self.init_clustering_model, self.init_classification_model)
        
        self.DM.update_data_item('df_news', self.RSS.df_news, concat=False)
        self.DM.update_data_item('df_viewed', pd.DataFrame(columns=self.RSS.df_news.columns), concat=False) # DF for viewed news
        self.DM.update_data_item('classifier', self.AGG.classifier, concat=False)
        self.DM.update_data_item('clusterizer', self.AGG.clusterizer, concat=False)
        return True
    
    def load_data_models(self):
        # try to load data and check if all data items in data_items_names in data_dict
        # set STATE ('ALL' - already fitted model, all data; 'NOT FITTED' - not fitted yet, some data; 'NEW' - no data)

        self.DM = Data_Manager(path_dict=self.path_dict)
        if len([i for i in self.data_items_names if i not in self.DM.data_dict.keys()]) == 0:
            return ('ALL')
        elif ('df_viewed' in self.DM.data_dict.keys()) and ('clust_weights' not in self.DM.data_dict.keys()):
            return 'NOT FITTED'
        else:
            return 'NEW'
        
    def select_news(self):
        categories = self.NFind.get_from_categories(n=self.n_from_cats)
        if self.STATE == 'ALL':
            interesting = self.NFind.get_interesting(self.AGG, n=self.n_interesting)
        else:
            interesting = self.NFind.df_unique_news.sample(self.n_interesting)
        return categories, interesting
    
    def check_viewed(self):
        # check and remove already viewed news from news DataFrame
        if self.STATE == 'ALL':
            viewed_id = np.hstack([self.DM.get_data_item('df_labeled').ID.values, self.DM.get_data_item('df_viewed').ID.values])
        elif self.STATE == 'NOT FITTED':
            viewed_id = self.DM.get_data_item('df_viewed').ID.values
        self.DM.update_data_item('df_news', self.RSS.df_news.query('ID not in @viewed_id'), concat=False)
        return True
    
    def start(self):
        #try to load data
        #if 'ALL' -> start existing
        #if 'NEW' -> call `create_new_system` and change STATE
        #if 'NOT FITTED' -> init aggregator with existing models
        # return DataFrames
        
        self.STATE = self.load_data_models()
        
        if self.STATE == 'NOT FITTED':
            self.AGG = Aggregator(clusterizer=self.DM.get_data_item('clusterizer'), 
                                  classifier=self.DM.get_data_item('classifier'))
            # filter already viewed news
            self.check_viewed()          
            
        elif self.STATE == 'ALL':
            self.DM.delete_old('df_labeled')    
            self.AGG = Aggregator(clusterizer=self.DM.get_data_item('clusterizer'), 
                                  classifier=self.DM.get_data_item('classifier'), 
                                  labeled_data=self.NVec.news2vec(self.DM.get_data_item('df_labeled').news.values), 
                                  labels=self.DM.get_data_item('df_labeled').label.values, 
                                  clust_weights=self.DM.get_data_item('clust_weights'))
            # filter already viewed news
            self.check_viewed()
        
        elif self.STATE == 'NEW':
            self.create_new_system()
            self.STATE = 'NOT FITTED'
        
        #initialize News_Finder
        self.NFind = News_Finder(self.DM.get_data_item('df_news'), News_Vectorizer(model=self.model))
        
        # get news DataFrames to show
        news_from_categories, news_interesting = self.select_news()
            
        return news_from_categories, news_interesting
    

    def handle_input(self, u_input): # the main method that GUI has to call
        ''' Call this method with `u_input` argument to communicate with the system.
            It takes a string `u_input` and returns appropriate output.
            
            Interactions as `u_input` -> `method output`:
            
            * 'upd' -> two pandas DataFrame objects (news in categories, interesting news)
            * 'exit' -> boolean: True, if data and models have been successfully saved, False otherwise
            * 'viewed' + ' ' + ID (e.g. 'viewed 005503512f38f130303cb133d656203b') -> two pandas DataFrame objects (news in categories, interesting news)
            * 'similar' + ' ' + ID (e.g. 'similar dc313bbf1bfca18d28e95862e972822f') -> pandas DataFrame with nearest news            
        '''
        # parse GUI input
        # call appropriate methods
        # return system output to GUI
        instruction = u_input.split()[0]
        if instruction == 'upd':
            #update news
            categories, interesting = self.update_news()
            return categories, interesting
        
        elif instruction == 'exit':
            is_saved = self.save()
            return is_saved
        
        elif instruction == 'viewed':
            #get news id and process viewed...
            n_id = u_input.split()[1:] # list, even if it consists of only one ID (generally)
            categories, interesting = self.handle_viewed(n_id)
            return categories, interesting

        elif instruction == 'similar':
            #get id and return similar
            n_id = u_input.split()[1] # string
            similar_news = self.find_similar(n_id)
            return similar_news

        else:
            return('unknown input')
        
    
    def update_news(self):
        # download news from rss feeds
        self.RSS = RSS_Feeds(self.feeds)
        
        # filter already viewed news
        self.check_viewed()
        
        # update news in News_Finder
        self.NFind.update_news(self.DM.get_data_item('df_news'))
        # return DataFrames
        news_from_categories, news_interesting = self.select_news()
            
        return news_from_categories, news_interesting
    
    def save(self):
        try:
            self.DM.save_model()
            return True
        except:
            return False
    
    def handle_viewed(self, n_id): #n_id : list
        # get news ID, check if already in viewed, remove from all news,.....
        viewed = self.NFind.df_unique_news.query('ID in @n_id')
        self.NFind.update_news(self.NFind.df_news.query('ID not in @n_id')) #filter already viewed and update data
        self.DM.update_data_item('df_viewed', viewed, concat=True)
        self.DM.update_data_item('df_news', self.NFind.df_news, concat=False)

        # if n(viewed) >= THRESHOLD -> update model -> save model
        if self.DM.get_data_item('df_viewed').shape[0] >= self.THRESHOLD:
            #update aggregator
            data = self.DM.get_data_item('df_viewed')
            colnames = data.columns
            self.AGG.update_aggregator(self.NVec.news2vec(data.news))
            
            #update df_labeled
            if self.STATE == 'ALL':
                data = pd.concat([self.DM.get_data_item('df_labeled').drop('label', axis=1), data])
            data['label'] = self.AGG.labels
            self.DM.update_data_item('df_labeled', data, concat=False)
            
            #update df_viewed (empty)
            self.DM.update_data_item('df_viewed', pd.DataFrame(columns=colnames), concat=False)
            
            #update data_items: classifier, clusterizer etc
            self.DM.update_data_item('classifier', self.AGG.classifier, concat=False)
            self.DM.update_data_item('clusterizer', self.AGG.clusterizer, concat=False)
            self.DM.update_data_item('clust_weights', self.AGG.clust_weights, concat=False)
            
            # update STATE
            self.STATE = 'ALL'
            
            #save model
            self.save()
            
        # return DataFrames
        news_from_categories, news_interesting = self.select_news()
        return news_from_categories, news_interesting
    
    
    def find_similar(self, n_id): #n_id : str
        # return the n nearest news to the given news item
        return self.NFind.get_similar(news_item_ID=n_id, n=self.n_similar)

In [9]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import display, HTML, clear_output
import webbrowser
import time

In [10]:
def create_item(value, placeholder='', description=''):
    w = widgets.HTML(
            value=value,
            placeholder=placeholder,
            description=description
        )
    return w

In [11]:
def view_similar(df):
    clear_output()
    display(widgets.HBox([UpdButton(), CloseButton()]))
    litems = [create_item('<b>'+str(i[1].title)+'</b>  <i>(' + str(i[1].datetime) + ')</i><br>'+str(i[1].summary) + '<br>') for i in df.iterrows()]
    ritems = [widgets.HBox([LinkButton(ID=i[1].ID, link=i[1].link), ShowSimilarButton(ID=i[1].ID)]) for i in df.iterrows()]
    items = []
    for i, j in zip(litems, ritems):
        items.append(i)
        items.append(j)
    display(widgets.GridBox(items, layout=widgets.Layout(grid='none / repeat(2)')))
    

In [12]:
def view_update(df_categories, df_interest):
    clear_output()
    print(time.ctime())
    display(widgets.HBox([UpdButton(), CloseButton()]))   
    cat_title = create_item('<br><h2>Interesting</h2>')
    litems = [create_item('<b>'+str(i[1].title)+'</b>  <i>(' + str(i[1].datetime) + ')</i><br>'+str(i[1].summary) + '<br>') for i in df_interest.iterrows()]
    ritems = [widgets.HBox([LinkButton(ID=i[1].ID, link=i[1].link), ShowSimilarButton(ID=i[1].ID)]) for i in df_interest.iterrows()]
    items = []
    for i, j in zip(litems, ritems):
        items.append(i)
        items.append(j)
    display(cat_title)
    display(widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2")))
    #display(widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 700px)")))
    #display(widgets.GridBox(items, layout=widgets.Layout(grid='none / repeat(1)')))
    
    for cat in df_categories.category.unique():
        df = df_categories.query('category == @cat')#.head(10)
        litems = [create_item('<b>'+str(i[1].title)+'</b>  <i>(' + str(i[1].datetime) + ')</i><br>'+str(i[1].summary) + '<br>') for i in df.iterrows()]
        ritems = [widgets.HBox([LinkButton(ID=i[1].ID, link=i[1].link), ShowSimilarButton(ID=i[1].ID)]) for i in df.iterrows()]
        cat_title = create_item('<br><h2>' + str(cat) + '</h2>')
        items = []
        for i, j in zip(litems, ritems):
            items.append(i)
            items.append(j)
        display(cat_title)
        #display(widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 700px)")))
        display(widgets.GridBox(items, layout=widgets.Layout(grid='none / repeat(2)')))

In [13]:
def upd_button_clicked(b):
    cat, interest = CI.handle_input('upd')
    view_update(cat, interest)

In [14]:
class LinkButton(widgets.Button):
    
    global CI
    global view_update
    
    def __init__(self, ID, link='https://www.google.com/', description='Read', disabled=False, style='', tooltip='Link to article', icon='link', *args, **kwargs):
        """Initialize the LinkButton class."""
        super(LinkButton, self).__init__(*args, **kwargs)
        # Create the button.
        self.link = link
        self.ID = ID
        self.description = description
        self.disabled = disabled
        #self.style = style
        self.tooltip = tooltip
        self.icon = "link"
        self.style.button_color = "lightblue"
        # Set on click behavior.
        self.on_click(self.process)

    def process(self, b):
        webbrowser.open(self.link)
        cat, interest = CI.handle_input('viewed '+ self.ID)
        view_update(cat, interest)


In [15]:
class UpdButton(widgets.Button):
    
    global CI
    global view_update
    
    def __init__(self, description='Update', disabled=False, style='', tooltip='Download news', icon='refresh', *args, **kwargs):
        """Initialize the LinkButton class."""
        super(UpdButton, self).__init__(*args, **kwargs)
        # Create the button.
        self.description = description
        self.disabled = disabled
        #self.style = style
        self.tooltip = tooltip
        self.icon = icon
        self.style.button_color = "lightgreen"
        # Set on click behavior.
        self.on_click(self.process)

    def process(self, b):
        cat, interest = CI.handle_input('upd')
        view_update(cat, interest)


In [20]:
class CloseButton(widgets.Button):
    
    global CI
    #global view_update
    
    def __init__(self, description='Close', disabled=False, style='', tooltip='Exit', icon='check', *args, **kwargs):
        """Initialize the LinkButton class."""
        super(CloseButton, self).__init__(*args, **kwargs)
        # Create the button.
        self.description = description
        self.disabled = disabled
        #self.style = style
        self.tooltip = tooltip
        self.icon = icon
        self.style.button_color = "gray"
        # Set on click behavior.
        self.on_click(self.process)

    def process(self, b):
        is_saved = CI.handle_input('exit')
        clear_output()


In [16]:
class ShowSimilarButton(widgets.Button):
    
    global CI
    global view_update
    
    def __init__(self, ID, description='Show similar', disabled=False, style='', tooltip='Find similar news', icon='search', *args, **kwargs):
        """Initialize the ShowSimilarButton class."""
        super(ShowSimilarButton, self).__init__(*args, **kwargs)
        # Create the button.
        self.ID = ID
        self.description = description
        self.disabled = disabled
        #self.style = style
        self.tooltip = tooltip
        self.icon = icon
        self.style.button_color = "thistle"
        # Set on click behavior.
        self.on_click(self.process)

    def process(self, b):
        similar = CI.handle_input('similar '+ self.ID)
        view_similar(similar)


In [17]:
CI = Communicator()

In [18]:
cat, interest = CI.start()

In [22]:
view_update(cat, interest)

Thu Jan 23 15:22:09 2020


HTML(value='<br><h2>Interesting</h2>', placeholder='')

GridBox(children=(HTML(value='<b>Labour leadership: Thornberry pressed on \'selective\' school choice</b>  <i>…

HTML(value='<br><h2>BBC News - Home</h2>', placeholder='')

GridBox(children=(HTML(value='<b>Coronavirus: \'Increased likelihood\' of cases in the UK</b>  <i>(Thu, 23 Jan…

...

GridBox(children=(HTML(value="<b>Wanted - volunteers to monitor Britain's growing slug population</b>  <i>(Thu…

HTML(value='<br><h2>BBC News - Technology</h2>', placeholder='')

GridBox(children=(HTML(value="<b>Twitter demands AI company stops 'collecting faces'</b>  <i>(Thu, 23 Jan 2020…

HTML(value='<br><h2>BBC News - Entertainment & Arts</h2>', placeholder='')

GridBox(children=(HTML(value='<b>Jessica Simpson reveals childhood sexual abuse</b>  <i>(Thu, 23 Jan 2020 09:5…